In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
ls data/sports_scores/

astros_1A10.csv*   Rice_10H80.csv*    Tex_15E40.csv*
dynamo_10H30.csv*  rockets_1A10.csv*  UH_10H50.csv*


In [3]:
ls data/crime_beat/

crime_10H30_10_17.csv*  crime_10H80_10_17.csv*  crime_1A10_10_17.csv*
crime_10H50_10_17.csv*  crime_15E40_10_17.csv*


In [4]:
games = 'data/sports_scores/Rice_10H80.csv'
crime = 'data/crime_beat/crime_10H80_10_17.csv'

# scores df

In [5]:
df = pd.read_csv(games,index_col='Date').sort_index(ascending=True)

## win column
df['win'] =  np.where(df['Pts'] > df['Opp'],1,0)

# only oposing name and win boolean
df = df[['Opponent','win']]

# show

df.head()

,Opponent,win
Date,,
2010-09-04,Texas,0
2010-09-18,Northwestern,0
2010-09-25,Baylor,0
2010-10-02,SMU,0
2010-10-16,Houston,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 2010-09-04 to 2017-11-25
Data columns (total 2 columns):
Opponent    45 non-null object
win         45 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


# Crime df

In [7]:
crime_df = pd.read_csv(crime).sort_index(ascending=True)

crime_df['Date_Hour'] = pd.to_datetime(crime_df['Date_Hour'])
crime_df['Date_Hour'] = crime_df['Date_Hour'].dt.date

crime_df = crime_df.set_index('Date_Hour')


# selectd columns
crime_df = crime_df[['OffenseType']]

# count Offense type given date
date_crimes = crime_df.groupby(crime_df.index)['OffenseType',].count()

## show
date_crimes.head()

,OffenseType
Date_Hour,
2010-01-03,4
2010-01-04,2
2010-01-05,3
2010-01-06,3
2010-01-07,1


In [8]:
#crime_df = pd.get_dummies(crime_df,drop_first=True)

In [9]:
#crime_df.info()

# Days Df
- create a df with everyday timestamp from Jan 2010 to Dec 2017

In [10]:
days = pd.date_range(start='01/01/2010', end='12/31/2017')
days = pd.DataFrame(days)

days.columns = ['days']
days = days.set_index('days').sort_index(ascending=True)
days.head()

""
days
2010-01-01
2010-01-02
2010-01-03
2010-01-04
2010-01-05


In [11]:
365 * 8

2920

In [12]:
len(days)/365

8.005479452054795

In [13]:
days.tail()

""
days
2017-12-27
2017-12-28
2017-12-29
2017-12-30
2017-12-31


# Merge days df with crimes_df

In [14]:
calendar_crimes = pd.merge(days,date_crimes,  left_index=True, right_index=True, how='left')
calendar_crimes.head()

,OffenseType
days,
2010-01-01,NaN
2010-01-02,NaN
2010-01-03,4.0
2010-01-04,2.0
2010-01-05,3.0


In [15]:
calendar_crimes.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 1 columns):
OffenseType    2632 non-null float64
dtypes: float64(1)
memory usage: 125.7 KB


In [16]:
calendar_crimes.OffenseType.value_counts(dropna=False)

 2.0     683
 1.0     619
 3.0     583
 4.0     342
NaN      290
 5.0     197
 6.0     119
 7.0      52
 8.0      26
 9.0       6
 10.0      4
 12.0      1
Name: OffenseType, dtype: int64

## Change nans with 0

In [17]:
calendar_crimes.OffenseType.replace( np.nan,0, inplace=True)

In [18]:
calendar_crimes.OffenseType.value_counts(dropna=False)

2.0     683
1.0     619
3.0     583
4.0     342
0.0     290
5.0     197
6.0     119
7.0      52
8.0      26
9.0       6
10.0      4
12.0      1
Name: OffenseType, dtype: int64

# merge calendar_crimes with score

In [19]:
merge_data = pd.merge(calendar_crimes,df,  left_index=True, right_index=True, how='left')

# change column names
#merge_data.columns = ['offenses','away_team','win']

merge_data.head()

,OffenseType,Opponent,win
days,,,
2010-01-01,0.0,NaN,NaN
2010-01-02,0.0,NaN,NaN
2010-01-03,4.0,NaN,NaN
2010-01-04,2.0,NaN,NaN
2010-01-05,3.0,NaN,NaN


In [20]:
merge_data['Opponent'].value_counts(dropna=False)

NaN                      2877
UTEP                        4
UTSA                        3
North Texas                 3
Louisiana Tech              3
Southern Mississippi        3
Florida Atlantic            2
Baylor                      2
SMU                         2
Marshall                    2
Army                        2
Tulane                      2
Old Dominion                1
Prairie View A&M            1
Kansas                      1
UAB                         1
East Carolina               1
Western Kentucky            1
UCLA                        1
Memphis                     1
Tulsa                       1
Hawaii                      1
Florida International       1
Purdue                      1
Wagner                      1
Charlotte                   1
Houston                     1
Texas                       1
Northwestern                1
Name: Opponent, dtype: int64

## Cleanup colums

In [21]:
merge_data.columns  = ['crime', 'away_team', 'won']

In [22]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null float64
away_team    45 non-null object
won          45 non-null float64
dtypes: float64(2), object(1)
memory usage: 171.3+ KB


In [23]:
merge_data.crime.value_counts(dropna=False)

2.0     683
1.0     619
3.0     583
4.0     342
0.0     290
5.0     197
6.0     119
7.0      52
8.0      26
9.0       6
10.0      4
12.0      1
Name: crime, dtype: int64

In [24]:
merge_data.crime = merge_data.crime.astype(int)

In [25]:
merge_data.crime.value_counts(dropna=False)

2     683
1     619
3     583
4     342
0     290
5     197
6     119
7      52
8      26
9       6
10      4
12      1
Name: crime, dtype: int64

In [26]:
merge_data

,crime,away_team,won
days,,,
2010-01-01,0,NaN,NaN
2010-01-02,0,NaN,NaN
2010-01-03,4,NaN,NaN
2010-01-04,2,NaN,NaN
2010-01-05,3,NaN,NaN
2010-01-06,3,NaN,NaN
2010-01-07,1,NaN,NaN
2010-01-08,0,NaN,NaN
2010-01-09,3,NaN,NaN


# Display non-nans

In [27]:
merge_data.dropna(subset=['away_team', 'won'])

,crime,away_team,won
days,,,
2010-09-04,1,Texas,0.0
2010-09-18,0,Northwestern,0.0
2010-09-25,1,Baylor,0.0
2010-10-02,3,SMU,0.0
2010-10-16,1,Houston,1.0
2010-11-20,2,East Carolina,1.0
2010-11-27,1,UAB,1.0
2011-09-10,4,Purdue,1.0
2011-10-08,4,Memphis,1.0


In [28]:
merge_data.to_csv('data/final/Rice1017_final.csv')

In [29]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2922 entries, 2010-01-01 to 2017-12-31
Data columns (total 3 columns):
crime        2922 non-null int64
away_team    45 non-null object
won          45 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 171.3+ KB
